In [1]:
import spacy
import pandas as pd
import requests
from urllib.parse import urljoin, urlencode
from spacy import displacy


In [2]:
nlp = spacy.load("en_core_web_sm")


In [3]:
def get_data(url,headers,**kwargs):

    """
    gets the data from a paginated rest api.
    """
    url_params = "?"+urlencode(kwargs)
    acctual_url = urljoin(url,url_params)
    response = requests.get(acctual_url,headers=headers)
    num_pages = response.json()["last_page"]
    data = []
    for page in range(1,num_pages +1):
        url_current = acctual_url + f"?page={page}"
        response = requests.get(url_current,headers=headers)
        data += response.json()["data"]["data"]

    flatten_data = [flatten_json(d) for d in data]
    return pd.DataFrame(flatten_data)
def flatten_json(y):
    """

    """
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        #elif type(x) is list:
        #    i = 0
        #    for a in x:
        #        flatten(a, name + str(i) + '_')
        #       i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [4]:
url = "http://develop.pk-db.com/api/v1/references/"
references = get_data(url, headers=None)


In [11]:
text =references["abstract"][1]

In [13]:
doc = nlp(text)
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

Eight CARDINAL
50 CARDINAL
more than 3 months DATE
25.9 years DATE
58.3 kg QUANTITY
8 CARDINAL
26.0 years DATE
55.5 kg QUANTITY
half CARDINAL
2.12 hours TIME
2.71 hours TIME
1.04 CARDINAL
0.96 CARDINAL
NS ORG
5.81 ml/min QUANTITY
4.12 ml QUANTITY
half CARDINAL


In [15]:
doc = nlp(text)
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)
displacy.render(doc, style="ent", jupyter=True)

In [41]:
from pathlib import Path
import PyPDF2

def get_pdf_stream(pdf_file):
    file_obj = open(pdf_file,'rb')
    reader = PyPDF2.PdfFileReader(file_obj)
    page_count = reader.numPages
    text = ''
    for page in range(page_count):
        page_obj = reader.getPage(page)
        text += page_obj.extractText()
    yield {'text': text}

In [42]:
path = "../../pkdb_data/studies/caffeine/Abernethy1985/Abernethy1985.pdf"



In [50]:

stream = get_pdf_stream(path)

In [51]:
doc = nlp(next(stream)["text"])
displacy.render(doc, style="ent", jupyter=True)
#for ent in doc.ents:
    # Print the entity text and its label
#    print(ent.text, ent.label_)